# Create data directory

In [2]:
pip -q install torchaudio librosa soundfile numpy matplotlib IPython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 68.3 MB/s eta 0:00:00


In [1]:
import os
import torchaudio

os.makedirs("./data", exist_ok=True)
torchaudio.datasets.LIBRISPEECH(root="./data", url="train-clean-100", download=True)


100%|██████████| 5.95G/5.95G [03:02<00:00, 34.9MB/s]


# Download LibriSpeech (train-clean-100)

In [3]:
dataset = torchaudio.datasets.LIBRISPEECH(root="./data", url="train-clean-100", download=False)

waveform, sample_rate, transcript, speaker_id, chapter_id, utterance_id = dataset[0]

print(f"Sample rate: {sample_rate}")
print(f"Transcript: {transcript}")
print(f"Waveform shape: {waveform.shape}")

Sample rate: 16000
Transcript: CHAPTER ONE MISSUS RACHEL LYNDE IS SURPRISED MISSUS RACHEL LYNDE LIVED JUST WHERE THE AVONLEA MAIN ROAD DIPPED DOWN INTO A LITTLE HOLLOW FRINGED WITH ALDERS AND LADIES EARDROPS AND TRAVERSED BY A BROOK
Waveform shape: torch.Size([1, 225360])


# Clean Dataset

In [5]:
import torchaudio
import torchaudio.transforms as T
import os
import json

def clean_data(min_duration=1.5, max_peak=0.95, sr=16000, save_dir="spectrogram_data"):
    dataset = torchaudio.datasets.LIBRISPEECH(root="./data", url="train-clean-100", download=False)
    # mel_transform = T.MelSpectrogram(sample_rate=sr, n_fft=512, hop_length=128, n_mels=64)

    cleaned = []

    if os.path.exists('meta/clean_metadata.json'):
        with open('meta/clean_metadata.json', 'r') as f:
            return json.load(f)

    os.makedirs(save_dir, exist_ok=True)
    os.makedirs("meta", exist_ok=True)


    for i in range(len(dataset)):
        waveform, sample_rate, _, speaker_id, _, utt_id = dataset[i]

        duration = waveform.shape[1] / sample_rate
        peak_val = waveform.abs().max().item()

        if duration >= min_duration and peak_val <= max_peak:
            waveform = waveform.mean(dim=0, keepdim=True)

            # Convert to spectrogram
            # spec = mel_transform(waveform)

            # filename = f"{speaker_id}_{utt_id}.pt"
            # path = os.path.join(save_dir, filename)
            # torch.save(spec, path)

            cleaned.append({
                "index": i,
                "duration": duration,
                "speaker_id": int(speaker_id),
                "utterance_id": int(utt_id)
                # "spectrogram_path": path
            })


    with open("meta/clean_metadata.json", "w") as f:
        json.dump(cleaned, f, indent=2)

    # print(f"✅ Converted and saved {len(cleaned)} spectrograms to '{save_dir}'")
    return cleaned


In [8]:
import torch

def create_data(num_samples, duration=5.0, seed=42):
    random.seed(seed)
    torch.manual_seed(seed)

    # Load dataset (no download here since we already have it)
    dataset = torchaudio.datasets.LIBRISPEECH(root="./data", url="train-clean-100", download=False)
    with open("meta/clean_metadata.json") as f:
        metadata = json.load(f)

    mel_transform = T.MelSpectrogram(
        sample_rate=16000,
        n_fft=1024,
        hop_length=256,
        n_mels=64
    )

    examples = []
    for _ in range(num_samples):
        while True:
            s1, s2 = random.sample(range(len(metadata)), 2)
            meta1, meta2 = metadata[s1], metadata[s2]
            if meta1["speaker_id"] != meta2["speaker_id"]:
                break

        wave1, sr, *_ = dataset[meta1["index"]]
        wave2, sr, *_ = dataset[meta2["index"]]

        max_len = int(duration * sr)
        wave1 = wave1[:, :max_len]
        wave2 = wave2[:, :max_len]

        # Pad if needed (ensure equal length)
        def pad(wave, target_len):
            if wave.shape[1] < target_len:
                pad_amt = target_len - wave.shape[1]
                wave = torch.nn.functional.pad(wave, (0, pad_amt))
            return wave

        wave1 = pad(wave1, max_len)
        wave2 = pad(wave2, max_len)

        # Mix the two: wave1 is target, wave2 is background
        mix = 0.8 * wave1 + 0.2 * wave2

        mix_spec = mel_transform(mix).squeeze(0)      # shape: [n_mels, T']
        target_spec = mel_transform(wave1).squeeze(0)  # shape: [n_mels, T']

        examples.append((mix_spec, target_spec))

    return examples

In [3]:
from torch import nn
class SplitterRNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, dropout):
    super(SplitterRNN, self).__init__()
    self.lstm = nn.LSTM(input_size=input_size,
                        hidden_size=hidden_size,
                        num_layers=num_layers,
                        batch_first=True,
                        dropout=dropout)
    self.linear = nn.Linear(hidden_size, input_size)

  def forward(self, x):
    x = x.transpose(1, 2)
    output, _ = self.lstm(x)  # output: (batch_size, time_steps, hidden_size)
    output = self.linear(output)  # output: (batch_size, time_steps, input_size)
    return output.transpose(1,2)

In [2]:
from torch.utils.data import Dataset, DataLoader
class SpectrogramDataset(Dataset):
    def __init__(self, examples):
        self.examples = examples

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        return self.examples[idx]

def collate_mel(batch):
    mix_specs, target_specs = zip(*batch)  # list of [n_mels, T_i]

    mix_padded = pad_sequence(mix_specs, batch_first=True)       # [B, T_max, n_mels]
    target_padded = pad_sequence(target_specs, batch_first=True) # [B, T_max, n_mels]

    # transpose to [B, n_mels, T] if your model expects that
    mix_padded = mix_padded.transpose(1, 2)       # [B, n_mels, T]
    target_padded = target_padded.transpose(1, 2) # [B, n_mels, T]

    return mix_padded, target_padded

In [9]:
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import random
import torchaudio
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

def simple_collate(batch):
    mix_specs, target_specs = zip(*batch)
    mix_batch = torch.stack(mix_specs)       # [B, n_mels, T]
    target_batch = torch.stack(target_specs) # [B, n_mels, T]
    return mix_batch.to(device=device), target_batch.to(device=device)

clean_data(min_duration=5.0)
constructed_dataset = create_data(10000, 5.0)
split = int(0.8 * len(constructed_dataset))
train_examples = constructed_dataset[:split]
test_examples = constructed_dataset[split:]

train_dataset = SpectrogramDataset(train_examples)
test_dataset = SpectrogramDataset(test_examples)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=simple_collate)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=True, collate_fn=simple_collate)

In [11]:
def train_rnn(model, criterion, optimizer, train_loader, nepochs=5000):
    losses = []
    model.train()
    for e in range(nepochs):
      total_loss = 0.0
      for mixes_batch, tgt_batch in train_loader:
          optimizer.zero_grad()
          output = model(mixes_batch)
          loss = criterion(output, tgt_batch)
          loss.backward()
          optimizer.step()

          total_loss += loss.item()
      avg_loss = total_loss / len(train_loader)
      losses.append(avg_loss)
      if e % 5 == 0:
        print(f"Epoch {e:2d} | Loss: {avg_loss:.6f}")
    return losses

In [15]:
# Parameters
input_size = 64
hidden_size = 128
num_layers = 2
batch_size = 8
sequence_length = 100
dropout = 0.25
lr = 0.001

model = SplitterRNN(input_size, hidden_size, num_layers, dropout).to(device)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)

train_rnn(model, criterion, optimizer, train_loader, nepochs=50)

Epoch  0 | Loss: 10380.914496
Epoch  5 | Loss: 8539.529343
Epoch 10 | Loss: 7461.977280
Epoch 15 | Loss: 6613.057019
Epoch 20 | Loss: 5929.655035
Epoch 25 | Loss: 5369.437668
Epoch 30 | Loss: 4917.615665
Epoch 35 | Loss: 4551.314039
Epoch 40 | Loss: 4239.506692
Epoch 45 | Loss: 3976.131297


[10380.914496459962,
 9754.39778149414,
 9393.959757202148,
 9084.584596069337,
 8800.050200622558,
 8539.529343383789,
 8296.778997528076,
 8068.640504882813,
 7855.19833190918,
 7653.193322814941,
 7461.977279907226,
 7277.634616394043,
 7099.590154968262,
 6929.778098388671,
 6768.10541015625,
 6613.057018981934,
 6464.515320983886,
 6322.100944702149,
 6184.706126159668,
 6053.892711395264,
 5929.655035217285,
 5808.644591979981,
 5693.68315826416,
 5583.822913848877,
 5475.264801391601,
 5369.437667541504,
 5269.90053704834,
 5175.854326293946,
 5086.006725830078,
 4999.3899889526365,
 4917.615664611816,
 4839.3344080200195,
 4763.3142830200195,
 4690.814848693848,
 4619.038354980469,
 4551.314039428711,
 4483.229152160645,
 4420.014795959472,
 4356.944166290284,
 4298.10276159668,
 4239.506692047119,
 4184.349056182861,
 4130.973856048584,
 4079.058049606323,
 4027.055926269531,
 3976.1312973327636,
 3926.381951828003,
 3883.24719039917,
 3835.8638897247315,
 3791.1066659240723]

In [16]:
def test_rnn(model, criterion, test_loader):
    model.eval()
    total_loss = 0.0

    with torch.no_grad():
        for mixes_batch, tgt_batch in test_loader:
            output = model(mixes_batch)
            loss = criterion(output, tgt_batch)
            total_loss += loss.item()

    avg_loss = total_loss / len(test_loader)
    print(f"Test Loss: {avg_loss:.6f}")
    return avg_loss


In [17]:
test_loss = test_rnn(model, criterion, test_loader)
test_loss

Test Loss: 3743.773630


3743.773629760742

# Designing Computational Experiments

Helper Functions

In [ ]:
pip install scipy sounddevice

In [ ]:
import sounddevice as sd
import numpy as np

# play_wav_file("path/to/your/audio.wav")
def play_audio(audio, sample_rate=44100):
    """
    Plays an audio sample.

    Parameters:
    - audio (numpy.ndarray): The audio data. Can be mono (1D) or stereo (2D).
    - sample_rate (int): Sampling rate in Hz (default: 44100).
    """
    if not isinstance(audio, np.ndarray):
        raise ValueError("Audio must be a NumPy array.")

    if audio.ndim > 2 or (audio.ndim == 2 and audio.shape[1] > 2):
        raise ValueError("Audio must be mono or stereo.")

    sd.play(audio, samplerate=sample_rate)
    sd.wait()  # Wait until audio playback is finished


import matplotlib.pyplot as plt

# visualize_wav("path/to/your/audio.wav")
def visualize_wav(filepath, show_spectrogram=True):
    """
    Visualizes the waveform (and optionally spectrogram) of a WAV file.

    Parameters:
    - filepath (str): Path to the WAV file.
    - show_spectrogram (bool): Whether to show the spectrogram below the waveform.
    """
    # Load audio
    data, samplerate = sf.read(filepath)
    duration = len(data) / samplerate
    time = np.linspace(0, duration, num=len(data))

    # Set up plot
    fig, axs = plt.subplots(2 if show_spectrogram else 1, 1, figsize=(12, 6), sharex=True)

    if not isinstance(axs, np.ndarray):
        axs = [axs]  # ensure axs is always a list for consistency

    # Plot waveform
    axs[0].plot(time, data)
    axs[0].set_title('Waveform')
    axs[0].set_ylabel('Amplitude')
    axs[0].grid(True)

    # Optional: Plot spectrogram
    if show_spectrogram:
        axs[1].specgram(data[:, 0] if data.ndim > 1 else data, Fs=samplerate, NFFT=1024, noverlap=512)
        axs[1].set_title('Spectrogram')
        axs[1].set_ylabel('Frequency [Hz]')
        axs[1].set_xlabel('Time [s]')

    plt.tight_layout()
    plt.show()

"""
Example:

# Load audio files
speaker1, sr = librosa.load("speaker1.wav", sr=None)
speaker2, _ = librosa.load("speaker2.wav", sr=None)

# Mix with speaker2 at 4x the volume of speaker1
mixed, s1_adj, s2_adj = adjust_loudness_mix(speaker1, speaker2, loudness_ratio=4.0)

# Save to disk
librosa.output.write_wav("mixed.wav", mixed, sr)
"""
def adjust_loudness_mix(speaker1_wave, speaker2_wave, loudness_ratio):
    """
    Adjusts the loudness of speaker2 relative to speaker1.

    Parameters:
        speaker1_wave (np.ndarray): Audio waveform of speaker 1.
        speaker2_wave (np.ndarray): Audio waveform of speaker 2.
        loudness_ratio (float): Factor to scale speaker2's amplitude.
                                 e.g., 2.0 means speaker2 is twice as loud as speaker1.

    Returns:
        mixed_wave (np.ndarray): The resulting mixture of the two speakers.
        adjusted_speaker2 (np.ndarray): The scaled version of speaker2_wave.
    """
    # Match length if needed
    min_len = min(len(speaker1_wave), len(speaker2_wave))
    speaker1_wave = speaker1_wave[:min_len]
    speaker2_wave = speaker2_wave[:min_len]

    # Apply loudness adjustment to speaker2
    adjusted_speaker2 = speaker2_wave * loudness_ratio

    # Mix the two signals
    mixed_wave = speaker1_wave + adjusted_speaker2

    # Optional: Normalize to avoid clipping
    max_val = np.max(np.abs(mixed_wave))
    if max_val > 1.0:
        mixed_wave = mixed_wave / max_val
        adjusted_speaker2 = adjusted_speaker2 / max_val
        speaker1_wave = speaker1_wave / max_val

    return mixed_wave, speaker1_wave, adjusted_speaker2

ModuleNotFoundError: No module named 'sounddevice'

# Nate does some delicious testing here

In [ ]:
# dataset = torchaudio.datasets.LIBRISPEECH(root="./data", url="train-clean-100", download=False)
waveform, sample_rate, transcript, speaker_id, chapter_id, utterance_id = dataset[0]

play_audio(waveform)

NameError: name 'play_audio' is not defined

**Experiment 1 - adjust loudness of target input and compare loss**

Goal: Assess how robust the model is when one speaker is significantly louder.

Design:

For each mixed sample, systematically vary the amplitude of one speaker:

 - Ratios: 1:1 (baseline), 2:1, 4:1, 8:1

Evaluate performance degradation or adaptation.

Hypotheses:

MSE may degrade linearly with imbalance.

CTC might maintain higher robustness if sequences are still distinguishable.

Experiment 2 - adjust loudness of background noise and compare loss (I think this may combine with experiments 1 and 4)

**Experiment 3 - compare efficiency on different languages**

Goal: Evaluate performance when one or both speakers speak a non-English language.

Design:

Use mixtures of English-English, English-Other (e.g., English-Spanish), and Other-Other.

Focus on tonal vs non-tonal languages for diversity.

Optional:

Augment training data with multilingual samples.

Try language embeddings if you're using a deeper pipeline.

Analysis:

Does the model performance degrade with unfamiliar phonetic structures?

Does it favor English content due to training bias?



**Experiment 4 - add non-human background noise and compare loss**

Goal: Evaluate robustness to noise interference (e.g., cafe noise, car engines, music).

Design:

Add non-human noise at varying SNRs (e.g., 30 dB, 20 dB, 10 dB).

Use both stationary and non-stationary noise.

Compare:

Performance with and without noise-aware preprocessing.

Try noise suppression front-ends or data augmentation with noise.

**Extra Experiments You Could Add**

Speaker permutation invariance: Does the model consistently output the same speaker when order varies?

Speaker gender and pitch: Investigate separation accuracy across gender combinations or pitch similarity.

Temporal length variation: Does it work on long conversations vs short utterances?

Window size sensitivity: Test how performance changes with different input chunk sizes.